# Objetivo:
Fazer um brute force pra entender qual o intervalo ideal de frequência pra maximizar as métricas de avaliação de detecção de beat, usando o espectrograma convencional e o mel-espectrograma 

In [ ]:
import glob
import csv
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import mir_eval
import pandas as pd

import IPython.display as ipd

In [ ]:
FS = 44100
N_FFT = 2048
HOP_LENGTH = 512
CHANNELS = [[0, 4, 8], [0, 6, 16], [0, 8, 16]]
METHODS = ["spectrogram", "melspectrogram"]
dataset_result = {}

In [ ]:
file_path = [i[:-4] for i in glob.glob('../datasets/candombe/*.wav')]

with open('onset_subbands_analysis.csv', 'w') as csvfile:
    columns = ['file', 'method', 'channels', 'num_channels', 'subband', 
               'f_score', 'p_score', 'cmlc', 'cmlt', 
               'amlc', 'amlt'
            ]
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    
    for file in file_path:
        print(f"processing {file}")
        x, fs = librosa.load(f"{file}.wav", mono=True, sr=FS)
        x_df = pd.read_csv(f"{file}.csv", names=["timestamp", "beat"])
        ground_truth = x_df['timestamp'].values
        
        for method in METHODS:
            for channels in CHANNELS:
                print(f'using {method} with channels = {channels}')
                beat_frames = []
                onset_frames = []
                detected_timestamps = []

                if method == "spectrogram":
                    X = np.abs(librosa.stft(x, n_fft=N_FFT, hop_length=HOP_LENGTH))
                    onset_subbands = librosa.onset.onset_strength_multi(S=X, sr=FS, channels=channels)
                else:
                    onset_subbands = librosa.onset.onset_strength_multi(y=x, sr=FS, channels=channels)
                    
                # ok, i KNOW that i can use only one for loop, but this is cropping my results and i 
                # DON'T wanna debug anything right now because it's 23h and i need to WORK tomorrow 
                # so YES i'll use three different for loops that loop in the SAME RANGE
                for i in range(len(onset_subbands)):
                    onset_frames.append(librosa.onset.onset_detect(onset_envelope=onset_subbands[i], sr=FS))

                for i in range(len(onset_subbands)):
                    _, beats = librosa.beat.beat_track(onset_envelope=onset_subbands[i], sr=FS)
                    beat_frames.append(beats)
                    timestamp = librosa.frames_to_time(beats, sr=FS)
                    detected_timestamps.append(timestamp)

                for i in range(len(onset_subbands)):
                    cmlc, cmlt, amlc, amlt = mir_eval.beat.continuity(ground_truth, detected_timestamps[i])

                    writer.writerow({
                        "file": file,
                        "method": method,
                        "channels": channels,
                        "num_channels": len(channels)-1,
                        "subband": i,
                        "f_score": mir_eval.beat.f_measure(ground_truth, detected_timestamps[i]),
                        "p_score": mir_eval.beat.p_score(ground_truth, detected_timestamps[i]),
                        "cmlc": cmlc,
                        "cmlt": cmlt,
                        "amlc": amlc,
                        "amlt": amlt
                    })

# análise

In [ ]:
df = pd.read_csv('onset_subbands_analysis.csv')

In [ ]:
df.groupby('method').mean()

In [ ]:
df.groupby('method').max()